# LabelMe 標註轉換為訓練格式
## 將 LabelMe JSON 轉為 Segmentation Mask 和 Pose Labels

In [ ]:
import json
import numpy as np
import cv2
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

print("✓ Libraries loaded")

In [ ]:
# 路徑設定
LABELME_DIR = Path('./output/rgb_for_labeling/rgb')  # LabelMe JSON 所在資料夾
OUTPUT_DIR = Path('./output/labelme_converted')

MASK_DIR = OUTPUT_DIR / 'masks'  # 人體分割 mask
LABELS_CSV = OUTPUT_DIR / 'pose_labels.csv'  # 姿勢標籤

MASK_DIR.mkdir(parents=True, exist_ok=True)

# 類別對應
CLASS_MAPPING = {
    'person_lying': {'id': 1, 'pose': 'lying'},
    'person_sitting': {'id': 2, 'pose': 'sitting'},
    'person_fallen': {'id': 3, 'pose': 'fallen'},
    'bed': {'id': 4, 'pose': None}  # 可選
}

print(f"LabelMe directory: {LABELME_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
def labelme_json_to_mask(json_path: Path, img_shape: tuple):
    """
    將 LabelMe JSON 轉為分割 mask
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    person_pose = None
    person_visible = False
    
    for shape in data.get('shapes', []):
        label = shape['label']
        points = shape['points']
        
        if label in CLASS_MAPPING:
            class_id = CLASS_MAPPING[label]['id']
            pose = CLASS_MAPPING[label]['pose']
            
            # 繪製 polygon
            pts = np.array(points, dtype=np.int32)
            cv2.fillPoly(mask, [pts], class_id)
            
            # 記錄姿勢
            if pose and not person_pose:
                person_pose = pose
                person_visible = True
    
    return mask, person_pose, person_visible

# 處理所有 JSON 檔案
json_files = sorted(LABELME_DIR.glob('*.json'))
print(f"\nFound {len(json_files)} LabelMe JSON files")

results = []

for json_path in tqdm(json_files, desc="Converting"):
    # 載入對應的圖片以取得尺寸
    img_path = json_path.with_suffix('.jpg')
    
    if not img_path.exists():
        continue
    
    img = cv2.imread(str(img_path))
    img_shape = img.shape
    
    # 轉換
    mask, pose, visible = labelme_json_to_mask(json_path, img_shape)
    
    # 儲存 mask
    mask_name = json_path.stem + '_mask.png'
    cv2.imwrite(str(MASK_DIR / mask_name), mask)
    
    # 記錄標籤
    results.append({
        'pair_id': json_path.stem,
        'image_path': str(img_path.relative_to(LABELME_DIR.parent)),
        'mask_path': str((MASK_DIR / mask_name).relative_to(OUTPUT_DIR)),
        'pose': pose if pose else 'none',
        'person_visible': 'yes' if visible else 'no'
    })

# 儲存標籤 CSV
labels_df = pd.DataFrame(results)
labels_df.to_csv(LABELS_CSV, index=False)

print(f"\n✓ Conversion complete!")
print(f"  Masks saved to: {MASK_DIR}")
print(f"  Labels saved to: {LABELS_CSV}")
print(f"\nPose distribution:")
print(labels_df['pose'].value_counts())

In [ ]:
# 視覺化檢查
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

sample_indices = np.random.choice(len(labels_df), min(9, len(labels_df)), replace=False)

for plot_idx, data_idx in enumerate(sample_indices):
    row = labels_df.iloc[data_idx]
    
    img_path = LABELME_DIR.parent / row['image_path']
    mask_path = OUTPUT_DIR / row['mask_path']
    
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    
    # 疊加 mask
    overlay = img_rgb.copy()
    overlay[mask > 0] = overlay[mask > 0] * 0.5 + np.array([255, 0, 0]) * 0.5
    
    axes[plot_idx].imshow(overlay.astype(np.uint8))
    axes[plot_idx].set_title(f"{row['pair_id']}\nPose: {row['pose']}", fontsize=9)
    axes[plot_idx].axis('off')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'conversion_check.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualization saved to: {OUTPUT_DIR / 'conversion_check.png'}")